# Music Recommendation System using Python

<p> <strong>Objective:</strong> Create a Music Recommendation System using the Spotify API and Python. This system leverages the power of Data Science to offer users personalized music recommendations based on their unique listening behavior and preferences. By analyzing extensive user data, including listening history, liked tracks, and user feedback, the system constructs individual user profiles to provide accurate and engaging music suggestions. </p>

<h3>Importing necessary libraries and the datasets.</h3>

In [57]:
import requests
import base64
from enum import Enum
import argparse
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.resetwarnings()

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

<p>We first need to have an access token. The access token serves as a temporary authorization credential, allowing the code to make authenticated requests to the Spotify API on behalf of the application
The CLIENT_ID and CLIENT_SECRET variables hold my credentials that uniquely identify the application making requests to the Spotify API. These credentials are obtained when a developer registers their application with Spotify’s developer dashboard. The Client ID identifies the application, while the Client Secret is a confidential key used for authentication.

</p>

In [58]:
# Replace with your own Client ID and Client Secret
CLIENT_ID = '83a3e0b2298b43d3bf59ddabf26e6e61'
CLIENT_SECRET = 'eb3a930592fe4a90b0d04fbc56e4c5ca'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}" #The client ID and secret are combined in the client_credentials variable
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token' #sends a POST request to the token_url
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials' #‘client_credentials’ to indicate that the application is requesting an access token for the client credentials flow.
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaning access token.")
    exit()

Access token obtained successfully.


In [59]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token,
    #Spotipy client to fetch information about the tracks in the specified playlist
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [60]:
playlist_id = '37i9dQZF1DX0XUfTFmNBRM' 

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

C:\Users\jms\anaconda3\lib\site-packages\spotipy\client.py:678: DeprecationWarning: You should use `playlist_items(playlist_id, ...,additional_types=('track',))` instead
  warnings.warn(


                                           Track Name  \
0                              Chaleya (From "Jawan")   
1                        Heeriye (feat. Arijit Singh)   
2   Janiye (from the Netflix Film "Chor Nikal Ke B...   
3   What Jhumka ? (From "Rocky Aur Rani Kii Prem K...   
4                              Not Ramaiya Vastavaiya   
5         Tere Vaaste (From "Zara Hatke Zara Bachke")   
6                  Chale Jaana Phir (Humko Tere Bina)   
7                                  Mahiye Jinna Sohna   
8   Tum Kya Mile (From "Rocky Aur Rani Kii Prem Ka...   
9                                        Malang Sajna   
10                                    Tere Pyaar Mein   
11                                  Jhoome Jo Pathaan   
12  Phir Aur Kya Chahiye (From "Zara Hatke Zara Ba...   
13            Tumhe Kitna Pyaar Karte (From "Bawaal")   
14  Pyaar Hota Kayi Baar Hai (From "Tu Jhoothi Mai...   
15                                    Zihaal e Miskin   
16                             

<p> We used a playlist ID: “37i9dQZF1DX0XUfTFmNBRM”. The code then calls the get_trending_playlist_data function to extract music data from the specified playlist using the provided access_token. The collected music data is stored in a DataFrame named music_df.</p>

In [61]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
     # Convert the release date to datetime object
    release_date = datetime.datetime.strptime(release_date, '%Y-%m-%d')


    # Calculate the time span between release date and today's date
    # Calculate the time span between release date and today's date
    time_span = datetime.datetime.now() - release_date


    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

<p>The above function takes the release date of a music track as input, which is provided in the format ‘YYYY-MM-DD’. It then uses the datetime.strptime function from the Python datetime module to convert the release date string to a datetime object. This conversion allows us to perform arithmetic operations with dates. </p>
<p>The weighted popularity score is computed based on the time span. The time_span.days attribute of the timedelta object gives the number of days in the time span between the release date and today. Adding 1 to the number of days ensures that the weight is never zero, even for very recent releases, as this would lead to a division by zero error.</p>

<p>The idea behind this formula is that the weight decreases as the time span between the release date and today increases. More recent releases will have a higher weight, while older releases will have a lower weight. As a result, when combining this weighted popularity score with other factors in a recommendation system, recent tracks will have a more significant impact on the final recommendations, reflecting users’ potential interest in newer music.</p>

<h3>Normalize the music features</h3>

In [62]:
#Normalize the music features using Min-Max scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [63]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    #This index will be used to compare the audio features of the input song with other songs in the dataset.
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

<p>The function calculates the similarity scores between the audio features of the input song and all other songs in the dataset. It uses cosine similarity, a common measure used in content-based filtering. The cosine_similarity function from scikit-learn is employed to compute these similarity scores.</p>
<p>The function identifies the num_recommendations most similar songs to the input song based on their audio features. It does this by sorting the similarity scores in descending order and selecting the top num_recommendations songs. </p>

In [69]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

<p>The hybrid approach aims to provide more personalized and relevant recommendations by considering both the content similarity of songs and their weighted popularity. The function takes input_song_name as the input, representing the name of the song for which recommendations are to be generated. The function first calls the content_based_recommendations function to get content-based recommendations for the input song. The num_recommendations parameter determines the number of content-based recommendations to be retrieved.
<br><br>
The function calculates the popularity score of the input song by retrieving the popularity value from the music_df DataFrame. It also calculates the weighted popularity score using the calculate_weighted_popularity function (previously defined) based on the release date of the input song. The alpha parameter controls the relative importance of content-based and popularity-based recommendations.</p>

<h3>Final function to generate music recommendations:</h3>

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
import datetime

input_song_name = "Jhoome Jo Pathaan"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Jhoome Jo Pathaan':
                                          Track Name  \
0                                       Malang Sajna   
4  Pyaar Hota Kayi Baar Hai (From "Tu Jhoothi Mai...   
1  What Jhumka ? (From "Rocky Aur Rani Kii Prem K...   
2             Character Dheela 2.0 (From "Shehzada")   
3                                  Tu Mile Dil Khile   

                                             Artists  \
0  Sachet Tandon, Parampara Tandon, Kumaar, Sache...   
4  Pritam, Arijit Singh, Amitabh Bhattacharya, Ch...   
1  Pritam, Arijit Singh, Jonita Gandhi, Ranveer S...   
2  Pritam, Amitabh Bhattacharya, Neeraj Shridhar,...   
3                             Asees Kaur, Stebin Ben   

                                          Album Name Release Date  Popularity  
0                                       Malang Sajna   2022-12-19        84.0  
4  Pyaar Hota Kayi Baar Hai (From "Tu Jhoothi Mai...   2023-02-10        79.0  
1  What Jhumka ? (From "Rocky Aur Ra

C:\Users\jms\AppData\Local\Temp\ipykernel_3028\209674984.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({
